In [3]:
import pandas as pd
import sys
import scanpy as sc
import numpy as np

In [7]:
obs

,guides_per_cell,guide_umi_counts,pct_counts_ribo,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,pool,sample_id,round1,well,biological_sample,Rep,plate
barcode,,,,,,,,,,,,,,
AAACATCGAACCGAGAGCAATTCT,5,203.0,0.440613,1027,870.000005,26.0,2.988506,eRZ58,eRZ58:eRZ58_GEX20A,GCAATTCT,A6,eRZ53_m2a,rep2,plate1
AAACATCGAACGCTTAACATTTGG,3,161.0,2.182540,135,188.999999,7.0,3.703704,eRZ58,eRZ58:eRZ58_GEX20A,ACATTTGG,H12,eRZ58_m2,rep2,plate1
AAACATCGAACGTGATACTTCATC,3,90.0,0.000000,88,221.999979,8.0,3.603604,eRZ58,eRZ58:eRZ58_GEX20A,ACTTCATC,E7,eRZ55_m9a,rep1,plate1
AAACATCGAACGTGATCATTTGGA,1,11.0,0.000000,128,233.000034,17.0,7.296136,eRZ58,eRZ58:eRZ58_GEX20A,CATTTGGA,D4,eRZ55_m5a,rep2,plate1
AAACATCGAATCCGTCCATTCTAC,3,40.0,0.000000,118,196.999978,3.0,1.522843,eRZ58,eRZ58:eRZ58_GEX20A,CATTCTAC,D12,eRZ55_m7b,rep1,plate1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTCACGCAGCTCGGTACATTCCTA,2,34.0,0.263193,1697,2532.999780,23.0,0.908014,eRZ58,eRZ58:eRZ58_GEX20A,CATTCCTA,A1,eRZ53_m1a,rep1,plate1
TTCACGCAGTCTGTCAGCTCTACT,3,43.0,0.273973,1422,729.999832,19.0,2.602740,eRZ58,eRZ58:eRZ58_GEX20A,GCTCTACT,D5,eRZ55_m5b,rep1,plate1
TTCACGCATCTTCACAATCCTTAC,2,97.0,0.000000,133,237.000014,22.0,9.282700,eRZ58,eRZ58:eRZ58_GEX20A,ATCCTTAC,C10,eRZ55_m3a,rep1,plate1


In [10]:
RESULT_PATH = "/oak/stanford/groups/engreitz/Users/ronghao/Sequencing/20251218_eRZ59_Split-seq_4M_Novaseq/perturb_pipeline/result/results_eRZ58_60C_analysis/"
ADATA_PATH = RESULT_PATH+"counts_filtered/adata.h5ad"
adata = sc.read_h5ad(ADATA_PATH)
adata
obs = adata.obs.copy()
print("AnnData loaded:", adata.shape)
print("\nobs columns preview:\n", list(obs.columns)[:50])
val_cols = ["guides_per_cell", "guide_umi_counts", "n_genes_by_counts", "total_counts"]

df_summary = (
    obs.groupby("biological_sample")
       .agg(
           # cell
           n_cells_per_sample=("biological_sample", "size"),
           n_cells_with_guides=("guides_per_cell", lambda x: int((x > 0).sum())),

           # ---------- GEX ----------
           gex_umis_per_sample=("total_counts", "sum"),
           gex_umis_per_cell_mean=("total_counts", "mean"),
           gex_umis_per_cell_median=("total_counts", "median"),
           genes_per_cell_mean=("n_genes_by_counts", "mean"),
           genes_per_cell_median=("n_genes_by_counts", "median"),

           #  ---------- gRNA ----------
           grna_umis_per_sample=("guide_umi_counts", "sum"),
           grna_umis_per_cell_mean=("guide_umi_counts", "mean"),
           grna_umis_per_cell_median=("guide_umi_counts", "median"),
           grna_detections_per_sample=("guides_per_cell", "sum"),
           grna_per_cell_mean=("guides_per_cell", "mean"),
           grna_per_cell_median=("guides_per_cell", "median"),
           frac_cells_with_guides=("guides_per_cell", lambda x: (x > 0).mean()),
           
           #  ---------- other ----------
           pct_mt_mean=("pct_counts_mt", "mean"),
           pct_mt_median=("pct_counts_mt", "median"),
           pct_ribo_mean=("pct_counts_ribo", "mean"),
           pct_ribo_median=("pct_counts_ribo", "median") 
       )
       .reset_index()
)
df_summary

df_summary.to_csv(RESULT_PATH+"qc_summary.csv", index=False)


AnnData loaded: (4779, 34183)

obs columns preview:
 ['guides_per_cell', 'guide_umi_counts', 'pct_counts_ribo', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'pool', 'sample_id', 'round1', 'well', 'biological_sample', 'Rep', 'plate']


/tmp/ipykernel_4284/3620333141.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  obs.groupby("biological_sample")
